In [1]:
import os
from lxml import etree
doc = etree.parse('../Data/xml_data/provenance.xml')
provenance = doc.findall('recordList/record/provenance/provenance.project.txt')
for item in provenance:
    if item.text == None:
        provenance.remove(item)
    else:
        continue

In [2]:
#Kunsthandelaren

#items with art dealers
dealer_items = list()
for elements in provenance:
    element = elements.text
    element_list = element.split(';')
    for elem in element_list:
        if 'Edward Speelman, London 1937' in elem:
            continue
        if 'on loan to the Dordrechts Museum' in elem:
            continue
        if elem.startswith(' dealer') or elem.startswith('dealer') or elem.startswith('Dealer') or elem.startswith(' Dealer'):
            dealer_items.append(elem)
        if 'art market' in elem:
            dealer_items.append(elem)

In [3]:
for item in dealer_items:
    test_item = item
    #art market dealer unknown
    if 'art market' in item:
        art_market = ''
        year = ''
        place_deal = ''
        buyer = ''
        city_buyer = ''
        dealer = ''
        city_dealer = ''
        source = ''
        items = item.split(',')
        #it is an art market, start of description
        art_market = items[0]
        art_market = art_market.strip()
        #is art market the only info?
        if len(items) ==1:
            proper_format = art_market
            readable_sent = f"Acquired on the {art_market}. "
        #year of dealing
        year_test = items[1].replace('.', '').replace(' ', '').replace('-','')
        if year_test.isalnum():
            year = items[1]
            year = year.strip()
        #place of dealing
        if items[1].isalpha():
            place_deal = item[1]
            place_deal = place_deal.strip()
        #buyer or dealer (dealer working for buyer)
        for item in items:
            if 'to' in item:
                buyer = item
                buyer = buyer.strip()
                buyer_index = items.index(item)
                #city of buyer
                city_buyer = items[buyer_index+1]
                city_buyer = city_buyer.strip()
            if 'by' and 'dealer' in item:
                dealer = item
                dealer = dealer.strip()
                dealer_index = items.index(item)
                #city of dealer
                city_dealer = items[dealer_index+1]
                city_dealer = city_dealer.strip()
        items = item.split('{')
        for item in items:
            if '{' in item:
                source = item.replace('{', '').replace('}', '').strip()
                source = "{" +source+"}"
        #form proper formatted provenance sentence
        proper_format = art_market
        if year != '':
            proper_format = f"{proper_format}, {year}"
        if place_deal != '':
            proper_format = f"{proper_format}, {place_deal}"
        if buyer != '':
            proper_format = f"{proper_format}, {buyer}"
        if city_buyer != '':
            proper_format = f"{proper_format}, {city_buyer}"
        if dealer != '':
            proper_format = f"{proper_format}, {dealer}"
        if city_dealer != '':
            proper_format = f"{proper_format}, {city_dealer}"
        if source != '':
            proper_format = f"{proper_format} {source}"
        #print('Proper format:', proper_format)
        #form readable provenance sentence
        readable_sent = ''
        if 'acquired on' in art_market:
            art_market = art_market.replace('acquired on the', '')
        if 'from the' in art_market:
            art_market = art_market.replace('from the', '')
        #Acquired on the art market in year, place, by dealer, place, sold to, place
        if art_market != '':
            readable_sent = f"Acquired on the {art_market}"
        if year != '':
            readable_sent = f"{readable_sent} in {year}"
        if place_deal != '':
            readable_sent = f"{readable_sent}, {place_deal}"
        if dealer != '':
            readable_sent = f"{readable_sent}, {dealer}"
        if city_dealer != '':
            readable_sent = f"{readable_sent}, {city_dealer}"
        if buyer != '':
            readable_sent = f"{readable_sent}, sold {buyer}"
        if city_buyer != '':
            readable_sent = f"{readable_sent}, {city_buyer}"
        readable_sent = f"{readable_sent}. "
       # print('Readable sentence:',readable_sent)
    
        parsed_dict = {'Source' : source, 'Year of sale' : year, 'Place of sale' : place_deal, 'Buyer' : buyer, 'City of buyer' : city_buyer, 'Dealer involved' : dealer, 'City of dealer' : city_dealer}
    #art market dealer known
    else:
        dealer_name = ''
        place = ''
        date = ''
        date2 = ''
        source = '' #add source to previous part as well, just in case -> Still need to do this!!
        dealer_cat_source = ''
        #source dealer catalogue
        itemsA = item.split('}')
        for item in itemsA:
            if 'cat.' in item:
                dealer_cat_source = item.strip()
        #name(s) of dealer(s)
        items = item.split(',')
        if items[0] == ' dealer':
            dealer_name = items[1].strip()
        elif items[0] == 'dealer':
            dealer_name = items[1].strip()
        elif items[0] == 'Dealer':
            dealer_name = items[1].strip()
        elif items[0] == ' Dealer':
            dealer_name = items[1].strip()
        else:
            for item in items:
                if (item.startswith('dealers') or item.startswith('Dealers') or item.startswith(' dealers') or item.startswith(' Dealers')) and not 'cat.' in item:
                    if len(item) == (7 or 8):
                        dealer_name = item[1]
                    if len(item) > 9:
                        dealer_name = item.replace('dealers', '').replace('Dealers', '').replace(',', '').strip()
                elif (item.startswith('dealer') or item.startswith('Dealer') or item.startswith(' dealer') or item.startswith(' Dealer')) and not 'cat.' in item:
                    if len(item) == (6 or 7):
                        dealer_name = item[1]
                    if len(item) > 8:
                        dealer_name = item.replace('dealer', '').replace('Dealer','').replace(',','').strip()
                elif (item.startswith('dealers') or item.startswith('Dealers') or item.startswith(' dealers') or item.startswith(' Dealers')) and 'cat.' in item:
                    dealer_item = item[1]
                    dealer_item = dealer_item.split('}')
                    dealer_name = dealer_item[1]
                    dealer_name = dealer_item.replace('dealers', '').replace('Dealers', '').replace(',','').strip()
                elif (item.startswith('dealer') or item.startswith('Dealer') or item.startswith(' dealer') or item.startswith(' Dealer')) and 'cat.' in item:
                    dealer_item = item[1]
                    dealer_item = dealer_item.split('}')
                    dealer_name = dealer_item[1]
                    dealer_name = dealer_item.replace('dealer', '').replace('Dealer', '').replace(',','').strip()     
        #year and place
        if items[0] == ' dealer' and len(items) > 2:
            date_test = items[2].replace('.', '').replace(' ', '').replace('-','')
            if date_test.isalnum() and not date_test.isalpha():
                date = items[2]
                date = date.strip()
            if date_test.isalpha():
                place = items[2].strip()
            if place != '' and len(items) >3:
                date = items[3].strip()
            if len(items) > 4:
                if date != '':
                    date_test2 = items[3].replace('.', '').replace(' ', '').replace('-','')
                    if date_test2.isalnum() and not date_test2.isalpha():
                        date2 = items[4]
                        date2 = date2.strip()
        else:
            date_test = items[1].replace('.', '').replace(' ', '').replace('-','')
            if date_test.isalnum() and not date_test.isalpha():
                date = items[1]
                date = date.strip()
            if date_test.isalpha():
                place = items[1].strip()
            if place != '' and len(items) >2:
                date = items[2].strip()
            if len(items) > 3:
                if date != '':
                    date_test2 = items[2].replace('.', '').replace(' ', '').replace('-','')
                    if date_test2.isalnum() and not date_test2.isalpha():
                        date2 = items[3]
                        date2 = date2.strip()
        #misc source
        items = item.split('{')
        for item in items:
            if '}' in item and 'cat' not in item:
                source = item.replace('}', '').strip()
                source = "{" +source+"}"
        #form proper formatted sentence
        #structure is '{catalogue source} dealer(s), dealer_name, place, date, date2, source
        proper_format = ''
        if dealer_cat_source != '':
            proper_format = "{" + dealer_cat_source +"} "
        if 'and' in dealer_name:
            proper_format = f"{proper_format}dealers"
        if 'and' not in dealer_name:
            proper_format = f"{proper_format}dealer"
        if dealer_name != '':
            proper_format = f"{proper_format}, {dealer_name}"
        if place != '':
            proper_format = f"{proper_format}, {place}"
        if date != '':
            proper_format = f"{proper_format}, {date}"
        if date2 != '':
            proper_format = f"{proper_format}, {date2}"
        if source != '':
            proper_format = f"{proper_format} {source}"
        #form proper readable provenance sentence
        #Structure: acquired by the dealer(s) dealer_name, place, date, date2
        readable_sent = 'Acquired by the dealer(s)'
        if dealer_name != '':
            readable_sent = f"{readable_sent} {dealer_name}"
        if place != '':
            readable_sent = f"{readable_sent} in {place}"
        if date != '':
            readable_sent = f"{readable_sent}, {date}"
        if date2 != '':
            readable_sent =f"{readable_sent}, {date2}"
        readable_sent = f"{readable_sent}."
        
    #test where mistakes are made
    if proper_format != test_item.strip():
        print('Old format: ',test_item)
        print('New format: ',proper_format, '\n')

Old format:   dealer Jacques Goudstikker, November 1920
New format:  dealer, Jacques Goudstikker, November 1920 

Old format:   dealers Benjamin and Nathan Katz, Dieren, 1937, 1938
New format:  dealers, Benjamin and Nathan Katz, Dieren, 1937, 1938 

Old format:   Dealer cat. Katz 1938, p. 22, no. 86.} dealer The Schaeffer Galleries (Benjamin and Nathan Katz's American agent), New York, 1937, 1938
New format:  {Dealer cat. Katz 1938, p. 22, no. 86.} dealers, The Schaeffer Galleries (Benjamin and Nathan Katz's American agent), New York, 1937, 1938 

Old format:   dealer F. Kleinberger, Paris, 1903
New format:  dealer, F. Kleinberger, Paris, 1903 

Old format:   dealers Benjamin and Nathan Katz, Dieren, 1937, 1938
New format:  dealers, Benjamin and Nathan Katz, Dieren, 1937, 1938 

Old format:   Dealer cat. Katz 1938, p. 20, no. 78.} dealer The Schaeffer Galleries (Benjamin and Nathan Katz's American agent), New York, 1938-47
New format:  {Dealer cat. Katz 1938, p. 20, no. 78.} dealers, T